In [1]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
from alibi.datasets import fetch_adult
from alibi.models.tensorflow import HeAE
from alibi.models.tensorflow import Actor, Critic
from alibi.models.tensorflow import ADULTEncoder, ADULTDecoder
from alibi.explainers.cfrl_base import Callback
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, get_statistics, \
    get_conditional_vector, apply_category_mapping
import random

def set_seed(s=0):
    random.seed(s)
    np.random.seed(s)
    tf.random.set_seed(s)






d:\ANACONDA\envs\dice_XAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logging
from typing import Optional, Tuple, Union
from sklearn.preprocessing import LabelEncoder
from alibi.utils.data import Bunch

logger = logging.getLogger(__name__)

df = pd.read_csv('E:\Graduation_Project\datasets\heart.csv')
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
df[categorical_features] = df[categorical_features].astype(str)


def fetch_heart(
    features_drop: Optional[list] = None, 
    return_X_y: bool = False, 
    categorical_features: Optional[list] = None
) -> Union[Bunch, Tuple[np.ndarray, np.ndarray]]:
    
    raw_data = df
    # get labels, features and drop unnecessary features
    labels = (raw_data['target'] == 1).astype(int).values

    if features_drop is None:
        features_drop = []
    features_drop = features_drop + ['target']

    data = raw_data.drop(features_drop, axis=1)
    features = list(data.columns)

    # 关键部分：外部指定优先，否则自动推断
    if categorical_features is None:
        categorical_features = [f for f in features if data[f].dtype == 'O']

    category_map = {}
    for f in categorical_features:
        le = LabelEncoder()
        data_tmp = le.fit_transform(data[f].values)
        data[f] = data_tmp
        category_map[features.index(f)] = list(le.classes_)

    # only return data values
    data = data.values
    target_names = [0, 1]

    if return_X_y:
        return data, labels

    return Bunch(
        data=data, 
        target=labels, 
        feature_names=features, 
        target_names=target_names, 
        category_map=category_map
    )

heart = fetch_heart(
    categorical_features=categorical_features
)


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# 1️⃣ 划分类别型和数值型特征


# Separate columns in numerical and categorical.
categorical_names = [heart.feature_names[i] for i in heart.category_map.keys()]
categorical_ids = list(heart.category_map.keys())

numerical_names = [name for i, name in enumerate(heart.feature_names) if i not in heart.category_map.keys()]
numerical_ids = [i for i in range(len(heart.feature_names)) if i not in heart.category_map.keys()]

# Split data into train and test
X, Y = heart.data, heart.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [4]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
# Define numerical standard scaler.
num_transf = MinMaxScaler()

# Define categorical one-hot encoder.
cat_transf = OneHotEncoder(
    categories=[range(len(x)) for x in heart.category_map.values()],
    handle_unknown="ignore"
)

# Define column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transf, numerical_ids),
        ("cat", cat_transf, categorical_ids),
    ],
    sparse_threshold=0
)

In [5]:
# Fit preprocessor.
preprocessor.fit(X_train)

# Preprocess train and test dataset.
X_train_ohe = preprocessor.transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

In [6]:
from tensorflow import keras
def build_simple_dnn():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(16, activation='relu', input_shape=(31,)))  # 输入31维特征
    model.add(keras.layers.Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
model = build_simple_dnn()
model.load_weights('my_model_weights.h5')   # 加载之前保存的权重

In [7]:
# Define prediction function.
predictor = lambda x: model.predict(preprocessor.transform(x), verbose=0)

predictor(X_test[0:2])

array([[0.84857166, 0.15142827],
       [0.88690937, 0.11309064]], dtype=float32)

In [8]:
from sklearn.metrics import accuracy_score
# Compute accuracy.
acc = accuracy_score(y_true=Y_test, y_pred=predictor(X_test).argmax(axis=1))
print("Accuracy: %.3f" % acc)

Accuracy: 0.836


In [9]:
numerical_names

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

In [10]:
# Define attribute types, required for datatype conversion.
feature_types = {'age': int, 'trestbps': int, 'chol': int, 'thalach': int, 'oldpeak': float}

# Define data preprocessor and inverse preprocessor. The invers preprocessor include datatype conversions.
heae_preprocessor, heae_inv_preprocessor = get_he_preprocessor(X=X_train,
                                                               feature_names=heart.feature_names,
                                                               category_map=heart.category_map,
                                                               feature_types=feature_types)

# Define trainset
trainset_input = heae_preprocessor(X_train).astype(np.float32)
trainset_outputs = {
    "output_1": trainset_input[:, :len(numerical_ids)]
}

for i, cat_id in enumerate(categorical_ids):
    trainset_outputs.update({
        f"output_{i+2}": X_train[:, cat_id]
    })
    
trainset = tf.data.Dataset.from_tensor_slices((trainset_input, trainset_outputs))
trainset = trainset.shuffle(1024).batch(32, drop_remainder=True)

In [11]:
# Define autoencoder path and create dir if it doesn't exist.
heae_path = os.path.join("tensorflow_heart", "ADULT_autoencoder")
if not os.path.exists(heae_path):
    os.makedirs(heae_path)

# Define constants.
EPOCHS = 50             # epochs to train the autoencoder
HIDDEN_DIM = 128         # hidden dimension of the autoencoder
LATENT_DIM = 20          # define latent dimension

# Define output dimensions.
OUTPUT_DIMS = [len(numerical_ids)]
OUTPUT_DIMS += [len(heart.category_map[cat_id]) for cat_id in categorical_ids]

# Define the heterogeneous auto-encoder.
heae = HeAE(encoder=ADULTEncoder(hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM),
            decoder=ADULTDecoder(hidden_dim=HIDDEN_DIM, output_dims=OUTPUT_DIMS))

# Define loss functions.
he_loss = [keras.losses.MeanSquaredError()]
he_loss_weights = [1.]

# Add categorical losses.
for i in range(len(categorical_names)):
    he_loss.append(keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    he_loss_weights.append(1./len(categorical_names))

# Define metrics.
metrics = {}
for i, cat_name in enumerate(categorical_names):
    metrics.update({f"output_{i+2}": keras.metrics.SparseCategoricalAccuracy()})
    
# Compile model.
heae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
             loss=he_loss,
             loss_weights=he_loss_weights,
             metrics=metrics)

if len(os.listdir(heae_path)) == 0:
    # Fit and save autoencoder.
    heae.fit(trainset, epochs=EPOCHS)
    heae.save(heae_path, save_format="tf")
else:
    # Load the model.
    heae = keras.models.load_model(heae_path, compile=False)

In [12]:
# 输出encoder的全部权重（包括隐藏层）
for i, weight in enumerate(heae.encoder.weights):
    print(f"参数{i}: {weight.name}, 形状: {weight.shape}")
    print(weight.numpy())  # 打印具体数值

参数0: adult_encoder_1/dense_22/kernel:0, 形状: (31, 128)
[[ 0.08873643  0.0709216  -0.17608166 ... -0.03643989  0.14825208
  -0.03980404]
 [ 0.09606059  0.0939892  -0.01512873 ... -0.11515966 -0.14141242
  -0.15575741]
 [ 0.11487781  0.0382008  -0.06492206 ... -0.12050369 -0.09879702
  -0.00706437]
 ...
 [ 0.14272259 -0.17787579 -0.06545106 ...  0.13873093 -0.20366322
  -0.11280332]
 [ 0.1729906   0.02108923  0.06432234 ...  0.22458002  0.00874016
  -0.09674485]
 [-0.08170243 -0.14469355  0.15328376 ...  0.03196003 -0.10160149
  -0.16604851]]
参数1: adult_encoder_1/dense_22/bias:0, 形状: (128,)
[ 0.05174459 -0.01869852  0.00749912  0.08547264  0.02847661  0.0195519
  0.05018203  0.05286779  0.05473476  0.07046813  0.03825403  0.03388041
  0.0450633   0.0452936   0.06294727  0.05906113 -0.02772297  0.01984996
 -0.01549751  0.02406473  0.04475469  0.03362958  0.0284313   0.02166346
  0.07286802  0.08531335 -0.00974583  0.04065216  0.00046673  0.01511034
  0.05609525  0.05653055  0.02226181  0.0

In [13]:
# Define constants
COEFF_SPARSITY = 0.5               # sparisty coefficient
COEFF_CONSISTENCY = 0.5            # consisteny coefficient
TRAIN_STEPS = 10000               # number of training steps -> consider increasing the number of steps
BATCH_SIZE = 100                   # batch size

In [14]:
import wandb


class RewardCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        
        if (step + update) % 100 != 0:
            return
        
        # get the counterfactual and target
        Y_t = sample["Y_t"]
        X_cf = model.params["decoder_inv_preprocessor"](sample["X_cf"])
        
        # get prediction label
        Y_m_cf = predictor(X_cf)
        
        # compute reward
        reward = np.mean(model.params["reward_func"](Y_m_cf, Y_t))
        wandb.log({"reward": reward})

class LossCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        # Log training losses.
        if (step + update) % 100 == 0:
            wandb.log(losses)

class TablesCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        # Log every 1000 steps
        if step % 1000 != 0:
            return
        
        # Define number of samples to be displayed.
        NUM_SAMPLES = 5
        
        X = heae_inv_preprocessor(sample["X"][:NUM_SAMPLES])        # input instance
        X_cf = heae_inv_preprocessor(sample["X_cf"][:NUM_SAMPLES])  # counterfactual
        
        Y_m = np.argmax(sample["Y_m"][:NUM_SAMPLES], axis=1).astype(int).reshape(-1, 1) # input labels
        Y_t = np.argmax(sample["Y_t"][:NUM_SAMPLES], axis=1).astype(int).reshape(-1, 1) # target labels
        Y_m_cf = np.argmax(predictor(X_cf), axis=1).astype(int).reshape(-1, 1)          # counterfactual labels
        
        # Define feature names and category map for input.
        feature_names = heart.feature_names + ["Label"]
        category_map = deepcopy(heart.category_map)
        category_map.update({feature_names.index("Label"): heart.target_names})
        
        # Construct input array.
        inputs = np.concatenate([X, Y_m], axis=1)
        inputs = pd.DataFrame(apply_category_mapping(inputs, category_map),
                              columns=feature_names)
        
        # Define feature names and category map for counterfactual output.
        feature_names += ["Target"]
        category_map.update({feature_names.index("Target"): heart.target_names})
        
        # Construct output array.
        outputs = np.concatenate([X_cf, Y_m_cf, Y_t], axis=1)
        outputs = pd.DataFrame(apply_category_mapping(outputs, category_map),
                               columns=feature_names)
        
        # Log table.
        wandb.log({
            "Input": wandb.Table(dataframe=inputs),
            "Output": wandb.Table(dataframe=outputs)
        })

In [15]:
# Define immutable features.
immutable_features = ['sex']

# Define ranges. This means that the `Age` feature can not decrease.
ranges = {'age': [0.0, 1.0]}

# Initialize wandb.
wandb_project = "Adult Census Counterfactual with Reinforcement Learning"
wandb.init(project=wandb_project)

explainer = CounterfactualRLTabular(predictor=predictor,
                                    encoder=heae.encoder,
                                    decoder=heae.decoder,
                                    latent_dim=LATENT_DIM,
                                    encoder_preprocessor=heae_preprocessor,
                                    decoder_inv_preprocessor=heae_inv_preprocessor,
                                    coeff_sparsity=COEFF_SPARSITY,
                                    coeff_consistency=COEFF_CONSISTENCY,
                                    category_map=heart.category_map,
                                    feature_names=heart.feature_names,
                                    immutable_features=immutable_features,
                                    ranges=ranges,
                                    train_steps=TRAIN_STEPS,
                                    batch_size=BATCH_SIZE,
                                    backend="tensorflow",
                                    callbacks=[LossCallback(), RewardCallback(), TablesCallback()])

wandb: Currently logged in as: a1227872681 (a1227872681-university-of-bristol) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:


# Fit the explainer.
explainer = explainer.fit(X=X_train)

# Close wandb.
wandb.finish()

100%|██████████| 10000/10000 [10:46<00:00, 15.48it/s]


actor_loss,█▄▄▃▂▅▂▄▄▅▄▂▄▄▁▂▃▄▄▄▃▃▃▃▃▃▃▄▃▄▄▃▄▃▃▂▄▄▂▄
consistency_loss,█▅▄▄▄▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▂▁▁▂▁▁▁▁▂▁▁▂▁▁▁▁▂
critic_loss,██▄▅▄▂▃▃▃▂▄▂▃▂▄▁▃▂▃▁▂▂▃▁▂▁▁▂▂▃▂▂▂▂▁▂▁▁▁▂
reward,▁█▅▆▇▇▇█▅▇▆▅▅▆▆▇▇▆▆▇▇▇▇▇▇▆▇▅▇▅▇▇▇▇▇▇▆▇▆▆
sparsity_cat_loss,▆█▆▇▅▃▃▄▄▅▄▃▄▃▄▂▃▃▂▃▂▃▄▄▃▄▃▂▂▂▂▁▁▃▃▂▂▂▂▃
sparsity_num_loss,█▇▅▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▂▂▁▁▂▁▁▁▁▂▂
actor_loss,-0.79029
consistency_loss,0.08625
critic_loss,0.02518
reward,0.79
sparsity_cat_loss,0.2475


In [17]:
# Generate counterfactual instances.
sample_num = 84
X = X_train[sample_num].reshape(1, -1)
Y_t = np.array([1 - Y_train[sample_num]])
C = [{
'age': [0, 20]       # 限定 age 在 0~70
}]
explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=4, batch_size=10)

1it [00:00,  6.42it/s]


In [18]:
X_cf = explanation.data['cf']['X']
X_cf[:, 9] = X_cf[:, 9].astype(float).round(1)
predictor(X_cf).argmax(axis=1)

array([1, 1, 1, 1], dtype=int64)

In [19]:
# Concat labels to the original instances.
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

# Concat labels to the counterfactual instances.
cf = np.concatenate(
    [X_cf, explanation.data['cf']['class']],
    axis=1
)

# Define new feature names and category map by including the label.
feature_names = heart.feature_names + ["target"]
category_map = deepcopy(heart.category_map)
category_map.update({feature_names.index("target"): heart.target_names})

# Replace label encodings with strings.
orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names
)

In [20]:
orig_pd.head(n=5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,54.0,1,4,140.0,239.0,0,0,160.0,0,1.2,1,0,normal,0.0


In [21]:
cf_pd['oldpeak'] = pd.to_numeric(cf_pd['oldpeak'], errors='coerce').round(1)
cf_pd.head(n=5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,54,1,4,140,234,0,0,159,1,1.4,2,2,reversible,1
1,54,1,4,140,234,0,0,159,1,1.4,2,2,reversible,1
2,54,1,4,141,233,0,0,158,1,1.3,1,3,reversible,1
3,54,1,4,141,233,0,0,159,1,1.4,2,2,reversible,1


In [22]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*has feature names.*')
import time

cfs = []


start = time.time()
for i, X_test_i in enumerate(X_test):
    X = X_test_i.reshape(1, -1)
    Y_t = 1 - predictor(X).argmax(axis=1)
    C = [{
    'age': [0, 20]       # 限定 age 在 0~70
    }]
    explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=4, batch_size=10)
    
    # Concat labels to the original instances.
    orig = np.concatenate(
        [explanation.data['orig']['X'], explanation.data['orig']['class']],
        axis=1
    )

    # Concat labels to the counterfactual instances.
    cf = np.concatenate(
        [explanation.data['cf']['X'], explanation.data['cf']['class']],
        axis=1
    )

    # Define new feature names and category map by including the label.
    feature_names = heart.feature_names + ["target"]
    category_map = deepcopy(heart.category_map)
    category_map.update({feature_names.index("target"): heart.target_names})

    # Replace label encodings with strings.
    orig_pd = pd.DataFrame(
        apply_category_mapping(orig, category_map),
        columns=feature_names
    )

    cf_pd = pd.DataFrame(
        apply_category_mapping(cf, category_map),
        columns=feature_names
    )
    cf_pd['oldpeak'] = cf_pd['oldpeak'].astype(float).round(1)
    cfs.append((
        orig_pd, cf_pd
    ))
end = time.time()

time_Alibi_cfrl = (end - start)/X_test.shape[0]

1it [00:00,  8.96it/s]
1it [00:00,  9.80it/s]
1it [00:00, 10.76it/s]
1it [00:00, 10.23it/s]
1it [00:00,  9.53it/s]
1it [00:00, 11.08it/s]
1it [00:00, 10.31it/s]
1it [00:00, 10.97it/s]
1it [00:00, 10.48it/s]
1it [00:00, 11.06it/s]
1it [00:00, 11.43it/s]
1it [00:00, 10.64it/s]
1it [00:00, 10.99it/s]
1it [00:00, 10.79it/s]
2it [00:00, 10.86it/s]
1it [00:00, 10.49it/s]
1it [00:00, 10.42it/s]
1it [00:00, 11.46it/s]
1it [00:00,  9.84it/s]
1it [00:00, 10.64it/s]
1it [00:00, 10.55it/s]
1it [00:00,  9.74it/s]
1it [00:00, 10.36it/s]
1it [00:00,  9.78it/s]
1it [00:00,  9.44it/s]
1it [00:00,  9.78it/s]
1it [00:00, 10.51it/s]
2it [00:00, 10.19it/s]
1it [00:00, 14.18it/s]
1it [00:00, 14.01it/s]
1it [00:00, 12.76it/s]
1it [00:00, 10.08it/s]
1it [00:00, 10.19it/s]
1it [00:00,  8.54it/s]
1it [00:00, 10.14it/s]
1it [00:00,  8.97it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.63it/s]
1it [00:00,  9.61it/s]
1it [00:00,  9.66it/s]
1it [00:00,  9.86it/s]
1it [00:00, 10.38it/s]
1it [00:00,  9.81it/s]
1it [00:00,

In [23]:
# 2. 映射
def custom_label_encoder(feature_names, df):
    arr = []
    for _, row in df.iterrows():
        encoded_row = []
        for i, f in enumerate(feature_names):
            if i in category_map:
                val = row[f]
                idx = category_map[i].index(val)
                encoded_row.append(idx)
            else:
                encoded_row.append(float(row[f]))
        arr.append(encoded_row)
    # 返回二维 array，类型 float32
    return np.array(arr)[:, :-1]


In [24]:
valid_count = 0
total = 0
for X_org, X_cfs in cfs:
    # 预测原始类别（X_org必须二维）
    y_true = model.predict(preprocessor.transform(custom_label_encoder(feature_names, X_org)), 
                           verbose=0).argmax(axis=1)[0]
    # 预测反事实类别（批量）
    y_cfs = model.predict(preprocessor.transform(custom_label_encoder(feature_names, X_cfs)), 
                          verbose=0).argmax(axis=1)
    # 统计类别发生变化的样本数
    valid_count += np.sum(y_cfs != y_true)
    total += len(y_cfs)
valid_alibi_cfrl = (valid_count / total if total > 0 else 0)

In [25]:
from XAI_metrics import calc_sparsity, calc_continuous_proximity, \
    calc_categorical_proximity, calc_manifold_distance, calc_cf_num


sparsity_alibi_cfrl = calc_sparsity(cfs, categorical_features)
con_proximity_alibi_cfrl = calc_continuous_proximity(cfs, numerical_names)
cat_proximity_alibi_cfrl = calc_categorical_proximity(cfs, categorical_features)
manifold_alibi_cfrl = calc_manifold_distance(cfs, df, categorical_features)
cf_num_alibi_cfrl = calc_cf_num(cfs)

In [ ]:
results_alibi_cfrl = {
    "method": ["Alibi_CFRL"],
    "Avg Time(s)": [time_Alibi_cfrl],
    "Validity": [valid_alibi_cfrl],
    "Sparsity": [sparsity_alibi_cfrl],
    "Proximity_con": [con_proximity_alibi_cfrl],
    "Proximity_cat": [cat_proximity_alibi_cfrl],
    "Manifold": [manifold_alibi_cfrl],
    "Avg CF count": [cf_num_alibi_cfrl]
}

df_results_alibi_cfrl = pd.DataFrame(results_alibi_cfrl)
df_results_alibi_cfrl = df_results_alibi_cfrl.round(2)

,method,Avg Time(s),Validity,Sparsity,Proximity_con,Proximity_cat,Manifold,Avg CF count
0,Alibi_CFRL,0.234381,0.995902,0.567062,0.362365,0.359631,9.748427,4.0


In [27]:
df_results_alibi_cfrl.to_csv('./results/Alibi_CFRL_result_heart.csv', index=False)